In [1]:
import findspark
findspark.init('/home/ronald/spark-2.4.5-bin-hadoop2.7')

In [2]:
# May cause deprecation warnings, safe to ignore, they aren't errors
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import desc

In [3]:
# Can only run this once. restart your kernel for any errors.
sc = SparkContext()

In [4]:
ssc = StreamingContext(sc, 10 )
sqlContext = SQLContext(sc)

In [5]:
socket_stream = ssc.socketTextStream("127.0.0.1", 9993)

In [6]:
lines = socket_stream.window( 20 )

In [7]:
from collections import namedtuple
fields = ("tag", "count" )
Tweet = namedtuple( 'Tweet', fields )

In [8]:
# Use Parenthesis for multiple lines or use \.
( lines.flatMap( lambda text: text.split( " " ) ) #Splits to a list
  .filter( lambda word: word.lower().startswith("#") ) # Checks for hashtag calls
  .map( lambda word: ( word.lower(), 1 ) ) # Lower cases the word
  .reduceByKey( lambda a, b: a + b ) # Reduces
  .map( lambda rec: Tweet( rec[0], rec[1] ) ) # Stores in a Tweet Object
  .foreachRDD( lambda rdd: rdd.toDF().sort( desc("count") ) # Sorts Them in a DF
  .limit(10).registerTempTable("tweets") ) ) # Registers to a table.

__________
### Now run TweetRead.py
__________

In [9]:
ssc.start()    

In [10]:
import time
from IPython import display
import matplotlib.pyplot as plt
import seaborn as sns
# Only works for Jupyter Notebooks!
%matplotlib inline 

In [30]:
count = 0
while count < 10:
    #print(count)
    time.sleep( 2 )
    top_10_tweets = sqlContext.sql( 'Select tag, count from tweets' )
    top_10_df = top_10_tweets.toPandas()
    display.clear_output(wait=True)
    sns.set()
    plt.figure( figsize = ( 10, 8 ) )
    #sns.barplot( x="count", y="tag", data=top_10_df)
    print(top_10_df)
    plt.show()
    count = count + 1

                   tag  count
0            #covidー19      7
1             #covid19      6
2               #covid      6
3    #laurentalexandre      3
4                  #tv      3
5         #covidー19…rt      3
6          #jogonegoro      2
7           #covid__19      2
8            #lockdown      2
9  #silivridekoronavar      2


<Figure size 720x576 with 0 Axes>

In [36]:
ssc.stop()